In [2]:
from langchain_google_genai import ChatGoogleGenerativeAI


An error occurred: module 'importlib.metadata' has no attribute 'packages_distributions'


c:\Users\rohit.MONDEE\miniconda3\envs\venv\lib\site-packages\google\api_core\_python_version_support.py:252: FutureWarning: You are using a Python version (3.9.25) past its end of life. Google will update google.api_core with critical bug fixes on a best-effort basis, but not with any other fixes or features. Please upgrade to the latest Python version, or at least Python 3.10, and then update google.api_core.
  warnings.warn(message, FutureWarning)


In [7]:
llm = ChatGoogleGenerativeAI(model="gemini-2.5-pro", temperature=0.6)

In [ ]:
import os
# Set the Google API key for Gemini
os.environ["GOOGLE_API_KEY"] = ""

In [10]:
from langchain.prompts import PromptTemplate

prompt_template = PromptTemplate(
    input_variables = ["country"],
    template = "what is the capital of {country}?"
)
prompt_template.format(country="India")

'what is the capital of India?'

In [11]:
from langchain.chains import LLMChain
chain = LLMChain(llm = llm, prompt = prompt_template)
print(chain.run("India"))

C:\Users\rohit.MONDEE\AppData\Local\Temp\ipykernel_31212\1932225214.py:2: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(llm = llm, prompt = prompt_template)
C:\Users\rohit.MONDEE\AppData\Local\Temp\ipykernel_31212\1932225214.py:3: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  print(chain.run("India"))


The capital of India is **New Delhi**.


# Combining multiple chains using simple Sequential data

In [12]:
capital_template = PromptTemplate(
    input_variables = ["country"],
    template = "what is the capital of {country}?"
)
capital_chain = LLMChain(llm = llm,prompt = capital_template)

famous_template = PromptTemplate(
    input_variables = ["capital"],
    template = "name some amazing places from the city {capital}"
)
famous_chain = LLMChain(llm = llm,prompt = famous_template)

In [13]:
capital_template = PromptTemplate(
    input_variables = ["country"],
    template = "what is the capital of {country}?"
)
capital_chain = LLMChain(llm = llm,prompt = capital_template, output_key="capital")

famous_template = PromptTemplate(
    input_variables = ["capital"],
    template = "name some amazing places from the city {capital}"
)
famous_chain = LLMChain(llm = llm,prompt = famous_template, output_key="famous_places")

In [14]:
from langchain.chains import SequentialChain
overall_chain = SequentialChain(
    chains = [capital_chain,famous_chain],
    input_variables = ["country"],
    output_variables = ["capital","famous_places"],
)

overall_chain({'country': 'India'})

C:\Users\rohit.MONDEE\AppData\Local\Temp\ipykernel_31212\1584197419.py:8: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  overall_chain({'country': 'India'})


{'country': 'India',
 'capital': 'The capital of India is **New Delhi**.',
 'famous_places': "Of course! New Delhi, the capital of India, is a city brimming with history, culture, and life. Here are some of its most amazing places, categorized for you:\n\n### Historical Marvels & Monuments\n\n1.  **India Gate:** A majestic war memorial standing at the heart of the city. It's a tribute to Indian soldiers who died in World War I.\n    *   **Why it's amazing:** The atmosphere here, especially in the evening when the monument is lit up and families gather on the surrounding lawns, is incredibly patriotic and vibrant.\n\n2.  **Qutub Minar:** This towering 73-meter-tall minaret is a UNESCO World Heritage Site. The intricate carvings and the surrounding archaeological complex are breathtaking.\n    *   **Why it's amazing:** It's a stunning example of early Indo-Islamic architecture, and the nearby Iron Pillar has baffled scientists for centuries because it has not rusted.\n\n3.  **Humayun's T

In [17]:
from langchain.schema import HumanMessage, AIMessage, SystemMessage

chatllm = ChatGoogleGenerativeAI(model="gemini-2.5-pro", temperature=0.6)

response = chatllm([
    SystemMessage(content = "You are a helpful assistant that helps people find information."),
    HumanMessage(content = "Who is the president of India?"),
])

C:\Users\rohit.MONDEE\AppData\Local\Temp\ipykernel_31212\1663430600.py:5: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = chatllm([


In [18]:
response.content

'The current president of India is **Droupadi Murmu**.\n\nShe assumed office on July 25, 2022, and is the 15th President of India. She is the first person from a tribal community and the second woman to hold the office.'

# prompt template + LLM + output parser

In [19]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.prompts.chat import ChatPromptTemplate
from langchain.schema import BaseOutputParser

In [20]:
class commaseperator(BaseOutputParser):
    def parse(self, text: str):
        return text.strip().split(',')

In [23]:
template = "you are helpful assistant when user gives input you should generate 10 words with comma seoerated"
human_template = "{input}"
chat_prompt = ChatPromptTemplate.from_messages([
    ("system", template),
    ("human" , human_template)
])




In [27]:
chain = chat_prompt|chatllm|commaseperator()
chain.invoke({"input":"generate programming languages"})

['Python',
 ' JavaScript',
 ' Java',
 ' C++',
 ' C#',
 ' Go',
 ' Rust',
 ' Swift',
 ' Kotlin',
 ' PHP']

ChatPromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='you are helpful assistant when user gives input you should generate 10 words with comma seoerated'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={})])
| ChatGoogleGenerativeAI(model='models/gemini-2.5-pro', google_api_key=SecretStr('**********'), temperature=0.6, client=<google.ai.generativelanguage_v1beta.services.generative_service.client.GenerativeServiceClient object at 0x00000255A314B8E0>, default_metadata=(), model_kwargs={})